# Web spam detection through link-based features

Project for Information Retrieval exam at University of Trieste, January 2021

In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import spam_detection as sd #python module containing custom functions
np.random.seed(2) #random seed for reproducibility

In the context of web search, spam is the fraudulent manipulation of web page content for the purpose of appearing high up in search results and web spam detection is a crucial issue for web search engines. In fact, ranking algorithms such as PageRank cannot explicitly penalize spam websites in favor of trustworthy ones, meaning that users may find very high-rank pages that have no useful content and are highly ranked only because they are part of a link farm, a popular way to fool ranking algorithms.

###  The dataset
The [WEBSPAM-UK2006](https://chato.cl/webspam/datasets/uk2006/) dataset contains 11402 hosts in the `.uk` domain, of which 7866 are labeled as spam or normal (non-spam). Newer datasets have been released by the same authors, but this 2006 version remains the one with the highest number of manually labeled samples.

The file `new_hostnames.csv` contains the names of the hosts in the dataset, while `webspam-uk2006-labels.txt` assigns to 8045 host names a label chosen among spam, normal or undecided. For the purpose of this project, undecided-labeled hosts were considered unlabeled, leaving only 7866 hosts labeled as spam or normal.

Finally, the file `uk-2006-05.hostgraph_weighted.txt` contains the weighted graph of the hosts, each row containing a host index, the indices of outlinked hosts and, for each of them, the number of outlinks.

The function `read_graph` returns a sparse `csr_matrix` $R^T$, with $R[i,j]$ equal to $0$ if there is no edge connecting host $i$ to host $j$ or to $\frac{1}{O[i]}$ where $O[i]$ is the total number of hosts outlinked by $i$.

Since PageRank algorithm requires $R$ to be a stochastic matrix, but there is no guarantee that each host has at least one outlink (dangling nodes problem), as proposed in [[1]](#references), an artificial node with a single self-loop was added to the graph, with ingoing edges from all dangling nodes.

In [2]:
hostnames=sd.hostnames_list('data/new_hostnames.csv')
labels_dict=sd.labels_dictionary('data/webspam-uk2006-labels.txt')
labels, labeled_dataset=sd.make_dataset(labels_dict,hostnames)
R_T=sd.read_graph('data/uk-2006-05.hostgraph_weighted.txt', len(hostnames))

### PageRank

In the following cell PageRank is computed iteratively, according to the equation:

$$
{rank}_{k+1}=\frac{\alpha}{N}\mathbf{1}+(1-\alpha)R^T\cdot {rank}_k
$$

where ${rank}_k$ is the column vector storing the PageRank scores at step $k$, $\mathbf{1}$ is a column vector of ones, $N$ is the number of nodes (in this case 11403) and $\alpha$ is the teleporting factor. The iterative computation is performed up to a fixed precision of $\epsilon$, i.e. until $|{rank}_k-{rank}_{k-1}|_1 < \epsilon$.


In [3]:
alpha=.1
eps=1e-8

print("PageRank computation")
rank=sd.compute_PR(alpha, eps, R_T)

PageRank computation
PageRank computed


Spam detection is particularly relevant for high PageRank hosts, since people tend to click on highly ranked pages, almost always within the first page of search engine results [[3]](#references).

However, we can see that PageRank alone is not able to filter out spam pages. In fact, if we restrict our view to the highest ranked 25% of the labeled dataset, 139 hosts out of 1966 are labeled as spam, as opposed to 773/7866 on the entire dataset. The proportion of spam hosts drops from 9.8% to 7.1%, but that is surely not enough to consider PageRank as a spam detection or spam-robust algorithm.

In [4]:
n=25 #percentage of labeled dataset to consider
nl=len(labeled_dataset)
labeled_top=(-rank[labeled_dataset]).argsort()[:(n*nl)//100]
y=labels
y_top=y[labeled_top]

print("Total spam hosts: ", sum(y))
print("Total labeled hosts: ", len(y))
print("Total spam hosts %: ", round(100*sum(y)/len(y), 3), '\n')
print("Spam hosts in top 25%: ", sum(y_top))
print("Labeled hosts in top 25%: ", len(y_top))
print("Spam hosts % in top 25%: ", round(100*sum(y_top)/len(y_top), 3))


Total spam hosts:  773
Total labeled hosts:  7866
Total spam hosts %:  9.827 

Spam hosts in top 25%:  139
Labeled hosts in top 25%:  1966
Spam hosts % in top 25%:  7.07


### An approximation for PageRank contributions

For the purpose of spam detection, it is useful to have a measure of the individual contributions of other hosts to the PageRank of a given host. This means obtaining a $N \times N$ matrix $PRM$ such that $PRM[i,j]$ is the contribution of node $i$ to the PageRank of node $j$ and such that the sum over the columns of $PRM$ is equal to the PageRank vector.

To do so, one possibility (explored in [[1]](#references)) is to use topic-specific PageRank $N$ times with jump vectors belonging to the canonical basis of $\mathbb{R}^N$. With this assumption, each row of $PRM$ satisfies the equation:
$$
PRM[u]=\alpha \mathbf{e_u}+(1-\alpha)PRM[u] \cdot R
$$
Where $\mathbf{e_u}$ is the $u$-th row vector of the canonical basis of $\mathbb{R}^N$.

Then, given a node index $v$, its corresponding contribution vector $cpr[v]$ is defined to be the $v$-th column of matrix $PRM$. This vector stores the contribution of all nodes to the PageRank of node $v$ and it will prove to be of particular interest when it comes to web spam detection.

However, computing topic-specific PageRank $N$ times is infeasible on large datasets, since each step requires an iterative computation conceptually identical to the one of PageRank.

To address this problem, the authors of [[2]](#references) propose a local algorithm for the computation of $\delta$-approximations of contribution vectors.

Given a node $v$ and its contribution vector $c_v:=cpr[v]$, a $\delta$-approximation of $c_v$ is a non-negative vector $c_v^*$ such that: 

$$
c_v[u]-\delta \cdot rank[v] \leq c_v^*[u] \leq c_v[u]\space \space \space \forall u \in \{0,...,N-1\}
$$

For each node the algorithm runs in time $O(\frac{1}{\alpha \delta})$. For $\alpha=0.1$, the value $\delta=0.001$ proved to be a good compromise between time efficiency and precision of approximation.

In [5]:
delta=1e-3
cstar=np.zeros((nl, len(rank)))
print("Approximation of contribution vectors for labeled hosts")

'''
approximate_contributions makes repeated access to the rows of R_T. Scipy sparse matrices are much slower than numpy arrays for
slicing, hence it is much more efficient to convert R_T to numpy for this computation. However, in the case of a larger dataset,
this step might not be feasible because of memory limitations.
'''

R_T_np=R_T.toarray()
for v in range(nl):
    print(str(v+1)+'/'+str(nl), end='\r')
    cstar[v]=(sd.approximate_contributions(labeled_dataset[v], alpha, delta*rank[labeled_dataset[v]], rank[labeled_dataset[v]], R_T_np))


Approximation of contribution vectors for labeled hosts


### Features for link-based web spam detection

There are two basic approaches to web spam detection: content-based and link-based.

In a content-based setting the `HTML` code of pages is scanned and some features are computed, usually related to text (for example, average word length or fraction of visible text). On the other hand, in a link-based setting (the one explored in this notebook), information is obtained exclusively from the web graph and computations performed over it (such as ranking algorithms).

Some very trivial link-based features that can be computed even before performing PageRank are indegree and outdegree, defined as follows:
 - Indegree: number of incoming links to a host
 - Outdegree: number of outgoing links from a host

Other useful features can be computed from the contribution vector of a node $v$ (or from a $\delta$-approximation of it):
 - Size of $\delta$-significant contributing set: this feature is defined as $cs\_size[v]=|S_{\delta}[v]|=|\{u : c_v^*[u]>\delta rank[v]\}|$.
 - Contribution from vertices in the $\delta$-significant contributing set: $cs\_contribution[v]=\frac{1}{rank[v]} \sum_{u \in S_{\delta}[v]} c_v^*[u]$ 
 - $l_2$ norm of $\delta$-significant contributing vector: $l_2\_norm[v]=\sum_{u \in S_{\delta}[v]} (\frac{c_v^*[u]}{rank[v]})^2$


These features can be used to train a binary classifier for spam detection.

In [6]:
x=np.zeros((nl, 5))
indegree, outdegree, cs_size, cs_contribution, l2_norm=sd.extract_features(R_T_np, delta, cstar, labeled_dataset, rank)
x[:,0]=indegree
x[:,1]=outdegree
x[:,2]=cs_size
x[:,3]=cs_contribution
x[:,4]=l2_norm

### Evaluation of classifiers

Some Machine Learning can be used to detect spam websites. Here three different approaches are compared: Logistic Regression, Decision Tree and Random Forest. For all these classifiers, the implementation from `scikit-learn` library is used with its default hyperparameter values, with the exception of the parameter `class_weight`, which is set to `balanced` to try to mitigate the unbalance in the dataset between spam and normal labeled data points. 

The fit and test of these models are performed through a 5-fold cross validation. Regarding performance metrics, in a classification setting accuracy is obviously to be taken into account but, in this case, it is not sufficient to compare and assess the models. In fact, given how unbalanced the dataset is, even a naive classifier that classifies all hosts as normal would achieve over 90% accuracy despite being completely useless for spam detection. To solve this problem, class-specific precision and recall statistics are taken into account for spam class. In particular, since spam labeling might be safety critical, one could imagine recall to be particularly relevant.

Tree-based classifiers work definitely better than Logistic Regression, scoring accuracies over 90% while attaining reasonable values for precision and recall on spam samples. However, Logistic Regression scores an outstanding 96% recall on spam meaning that it succeeds in retrieving almost all spam hosts (at the cost of many false alarms, captured by low precision score).

In [7]:
k=5 #k-fold CV parameter

print("Logistic Regression")
clf=LogisticRegression(class_weight='balanced')
sd.print_prediction_metrics(clf, x, y, k)

print("\nDecision Tree")
clf=DecisionTreeClassifier(class_weight='balanced')
sd.print_prediction_metrics(clf, x, y, k)

print("\nRandom Forest")
clf=RandomForestClassifier(class_weight='balanced')
sd.print_prediction_metrics(clf, x, y, k)

Logistic Regression
Accuracy:  0.71
Precision on spam:  0.246
Recall on spam:  0.96

Decision Tree
Accuracy:  0.91
Precision on spam:  0.558
Recall on spam:  0.554

Random Forest
Accuracy:  0.93
Precision on spam:  0.698
Recall on spam:  0.554


Once again, it is particularly interesting to restrict the view to the top 25% highest ranked labeled hosts, since these are the ones that users are most likely to run into.

Here there is no huge gap in accuracy between the three classifiers, with values well over 90% for all of them. Tree-based methods confirm their higher precision on spam samples but, because of the different misclassification cost between the two class and because of a (almost) perfect recall score on spam, Logistic Regression may be preferrable.

In [8]:
x_top=x[labeled_top]

print("Logistic Regression")
clf=LogisticRegression(class_weight='balanced')
sd.print_prediction_metrics(clf, x_top, y_top, k)

print("\nDecision Tree")
clf=DecisionTreeClassifier(class_weight='balanced')
sd.print_prediction_metrics(clf, x_top, y_top, k)

print("\nRandom Forest")
clf=RandomForestClassifier(class_weight='balanced')
sd.print_prediction_metrics(clf, x_top, y_top, k)

Logistic Regression
Accuracy:  0.94
Precision on spam:  0.552
Recall on spam:  0.993

Decision Tree
Accuracy:  0.96
Precision on spam:  0.754
Recall on spam:  0.683

Random Forest
Accuracy:  0.98
Precision on spam:  0.831
Recall on spam:  0.813


### Robust PageRank

PageRank contributions can be also used to define a robust version of PageRank, i.e. a ranking algorithm similar to PageRank that penalizes spam pages.

From the definition of the matrix $PRM$:

$$
rank[v]=\sum_u PRM[u,v]
$$

Robust PageRank can be defined by the following equation:

$$
robust\_rank[v]=\sum_u min(PRM[u,v],\delta \cdot rank[v])
$$

The intuition behind this definition is that spam pages tend to have a huge PageRank contribution from a very small set of nodes, while trustworthy pages have lower individual contributions but from more nodes. Then, this definition penalizes spam pages by eliminating very high values for $PRM[u,v]$ and substituting them with $\delta \cdot rank[v]$.

Robust PageRank can be rewritten as: 

$$robust\_rank[v]=\sum_{u \notin S_\delta[v]} PRM[u,v]+\sum_{u \in S_\delta[v]}\delta \cdot rank[v]=\sum_u PRM[u,v] - \sum_{u \in S_\delta[v]} PRM[u,v] +\sum_{u \in S_\delta[v]}\delta \cdot rank[v]=
$$
$$
=rank[v]- \sum_{u \in S_\delta[v]} PRM[u,v]+\delta \cdot rank[v] \cdot cs\_size[v]
$$

$PRM[u,v]$ can be approximated by $c_v^*[u]$ and recalling the definition:

$$
cs\_contribution[v]=\frac{1}{rank[v]} \sum_{u \in S_{\delta}[v]} c_v^*[u] 
$$
It follows that:
$$
robust\_rank[v]=rank[v]- cs\_contribution[v] \cdot rank[v]+\delta \cdot rank[v] \cdot cs\_size[v]
$$


In [9]:
robust_rank=rank[labeled_dataset]-np.multiply(rank[labeled_dataset], cs_contribution)+delta*np.multiply(rank[labeled_dataset], cs_size)

With this modified ranking algorithm, spam hosts almost disappear from the top 25% of the labeled dataset, with 3 spam samples over 1966. This means that only 0.15% of the first 1966 ranked hosts are spam, compared to 9.8% over the entire labeled dataset.

Another important point is that Robust PageRank returns (for non-spam pages) an ordering similar to the one given by PageRank. A simple way to verify this feature is by intersecting the top 25% sets obtained with Normal and Robust PageRank algorithms. These sets (apart from spam hosts) are almost overlapping. 1717 hosts out of 1966 are present in both sets.

In [10]:
robust_labeled_top=(-robust_rank).argsort()[:(n*nl)//100]
robust_y_top=y[robust_labeled_top]

print("Total spam hosts: ", sum(y))
print("Total labeled hosts: ", len(y))
print("Total spam hosts %: ", round(100*sum(y)/len(y), 3), '\n')
print("Spam hosts in top 25% (Robust PageRank): ", sum(robust_y_top))
print("Labeled hosts in top 25% (Robust PageRank): ", len(robust_y_top))
print("Spam hosts % in top 25% (Robust PageRank): ", round(100*sum(robust_y_top)/len(robust_y_top), 3), '\n')

print("Size of intersection between top 25% with Normal and Robust PageRank: ", len(np.intersect1d(robust_labeled_top, labeled_top)))

Total spam hosts:  773
Total labeled hosts:  7866
Total spam hosts %:  9.827 

Spam hosts in top 25% (Robust PageRank):  3
Labeled hosts in top 25% (Robust PageRank):  1966
Spam hosts % in top 25% (Robust PageRank):  0.153 

Size of intersection between top 25% with Normal and Robust PageRank:  1717


### References
<a id='references'></a>
- [1] R. Andersen, C. Borgs, J. Chayes, J. Hopcroft, K. Jain, V. Mirrokni and S. Teng. [Robust PageRank and locally computable spam detection features](https://dl.acm.org/doi/10.1145/1451983.1452000). In Proceedings of the 4th international workshop on Adversarial information retrieval on the web (AIRWeb '08), 2008.
- [2] R. Andersen, C. Borgs, J. Chayes, J. Hopcroft, V. Mirrokni and S. Teng. [Local computation of PageRank contributions](https://link.springer.com/chapter/10.1007/978-3-540-77004-6_12). In 5th International Workshop of Algorithms and Models for the Web-Graph, 2007.
- [3] C. Barry, M. Lardner. [A Study of First Click Behaviour and User Interaction on the Google SERP](https://link.springer.com/chapter/10.1007/978-1-4419-9790-6_7). In: Pokorny J. et al. (eds) Information Systems Development. Springer, New York, 2011.